In [ ]:
import os                          # 파일 및 디렉토리 경로 확인용 모듈
from ultralytics import YOLO       # YOLO 모델 관련 라이브러리 (사용 여부에 따라 삭제 가능)
import shutil                      # 파일 및 폴더 삭제를 위한 모듈

# 삭제할 폴더 경로를 지정합니다.
folder_path = "checkpoints/train"

# 지정한 폴더가 실제로 존재하는지 확인합니다.
if os.path.exists(folder_path):
    # 폴더가 존재하면, 폴더와 그 안의 모든 내용을 삭제합니다.
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")  # 삭제 완료 메시지 출력
else:
    # 폴더가 존재하지 않으면, 해당 메시지를 출력합니다.
    print(f"Folder '{folder_path}' does not exist.")

In [ ]:
# 하이퍼파라미터 설정
EPOCH = 100        # 총 학습 에포크 수
BATCH_SIZE = 8     # 배치 크기

In [ ]:
# 로컬에 다운로드된 YOLOv9n 모델 불러오기
# - "yolov9n.pt"는 YOLOv9의 가장 작은 버전(nano) 모델입니다.
# - 사전에 다운로드된 가중치 파일(.pt)을 사용하여 모델을 초기화합니다.
model = YOLO("yolov9n.pt")

In [ ]:
from ultralytics import YOLO                         # YOLOv8 모델을 불러오기 위한 Ultralytics 라이브러리
from ultralytics.utils import LOGGER                 # Ultralytics의 내부 로거(LOGGER)를 사용
import logging
import torch                                          # PyTorch 라이브러리
import matplotlib.pyplot as plt                       # 실시간 손실 그래프 시각화용
import numpy as np
import IPython.display as display                     # IPython 환경에서 출력 클리어에 사용

LOGGER.propagate = False                              # Ultralytics의 로거 메시지 중복 방지를 위해 propagate 비활성화

# 🔹 손실값 저장용 리스트 초기화
epochs = []           # 에폭 번호 저장
box_losses = []       # 박스(객체 위치) 손실 저장
cls_losses = []       # 분류(classification) 손실 저장
dfl_losses = []       # DFL (Distribution Focal Loss) 손실 저장

# 🔸 에폭이 끝날 때마다 호출되는 콜백 함수 정의
# - 현재 에폭에서 계산된 손실값을 추적 및 그래프로 출력
def loss_logger(trainer):
    if hasattr(trainer, "tloss") and trainer.tloss is not None:
        try:
            # 손실값이 Tensor일 경우 리스트로 변환
            if isinstance(trainer.tloss, torch.Tensor):
                loss_values = trainer.tloss.tolist()
            else:
                loss_values = [trainer.tloss]  # float일 경우 리스트로 래핑

            # 손실값 분리 (YOLO는 일반적으로 box, cls, dfl 3개 사용)
            if len(loss_values) >= 3:
                box_loss, cls_loss, dfl_loss = loss_values[:3]
            elif len(loss_values) == 2:
                box_loss, cls_loss = loss_values[:2]
                dfl_loss = 0
            else:
                box_loss = loss_values[0]
                cls_loss, dfl_loss = 0, 0

            # Jupyter 환경에서 출력 클리어
            display.clear_output(wait=True)

            # 그래프를 위한 손실값 저장
            epochs.append(trainer.epoch + 1)
            box_losses.append(box_loss)
            cls_losses.append(cls_loss)
            dfl_losses.append(dfl_loss)

            # 현재 에폭 손실값 출력
            print(f'[에폭 {trainer.epoch + 1}] 박스 손실: {box_loss:.3f} | '
                  f'분류 손실: {cls_loss:.3f} | DFL 손실: {dfl_loss:.3f}')

            # 실시간 손실 그래프 시각화
            plt.figure(figsize=(10, 5))
            plt.plot(epochs, box_losses, marker='o', linestyle='-', label='Box Loss', color='b', alpha=0.7)
            plt.plot(epochs, cls_losses, marker='s', linestyle='-', label='Cls Loss', color='g', alpha=0.7)
            plt.plot(epochs, dfl_losses, marker='^', linestyle='-', label='DFL Loss', color='r', alpha=0.7)

            # 그래프 스타일 설정
            plt.title('YOLO Training Loss per Epoch')
            plt.xlabel('Epoch')
            plt.ylabel('Loss Value')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Loss logging error: {e}")

# 🔹 콜백 등록
# - 각 에폭이 끝날 때마다 loss_logger 함수를 실행
model.add_callback("on_train_epoch_end", loss_logger)

# 🔸 모델 학습 시작
results = model.train(
    data="data.yaml",         # 학습 데이터 구성 정의 파일
    epochs=EPOCH,               # 총 학습 에폭 수
    batch=BATCH_SIZE,         # 배치 크기
    imgsz=320,                # 입력 이미지 크기
    project="checkpoints",    # 결과 저장 디렉토리
    save=True,                # 모델 체크포인트 저장 여부
    exist_ok=True,            # 기존 디렉토리 덮어쓰기 허용
    save_period=10,           # 몇 에폭마다 모델 저장할지 설정
    plots=False               # 기본 내장 그래프 저장 비활성화 (우리는 직접 그림)
)